# Plotting NOAA's nClimGrid Daily Stored in AWS
## Written By Jared Rennie

Ever see a plot like this?

<img src='nclimgrid-conus_tmax_20210724.png' width='615' height='463' alt='so cool!'>

And wanted to make one yourself? Wonder where the data came from? Are you sick and tired of seeing "Made by @jjrennie" and wanted to plot something better? Well now is your chance!

### What You Need

First off, the entire codebase works in Python... sort of. 

You will need the following programs installed: 
- Python
- s3fs | numpy | xarray | geopandas | rioxarray | matplotlib | cartopy | shapely | metpy | h5netcdf | h5py 
    
The "easiest" way is to install these is by installing <a href='https://www.anaconda.com/' target="_blank">Anaconda</a>, and then applying <a href='https://conda-forge.org/' target="_blank">conda-forge</a>. Afterward, then you can install the above packages. 

### Importing Packages
Assuming you did the above, it should (in theory) import everything no problem:

In [ ]:
# Import Packages
import sys, time, datetime, math, calendar
import s3fs

import numpy as np
import xarray as xr
import geopandas as gpd
import rioxarray

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
from cartopy.feature import ShapelyFeature
from shapely.geometry import mapping
from metpy.plots import USCOUNTIES

### Choose a date and element

If you made it this far, great!

<a href='https://www.ncei.noaa.gov/products/land-based-station/nclimgrid-daily' target='blank'>NOAA's nClimGrid Daily </a> is a gridded, 5km dataset of daily temperature (max,min,avg) and precipitation for the Contiguous United States going back to 1951. There are gridded netCDF versions, and also values aggregated to certain regions (county, state, NWS Forecast Office). If you ever wanted to investigate the data, you can check them out at these locations:

- **WWW:** <a href='https://www.ncei.noaa.gov/pub/data/daily-grids/v1-0-0/' target = 'blank'>https://www.ncei.noaa.gov/pub/data/daily-grids/v1-0-0/</a>
- **AWS:** s3://noaa-nclimgrid-daily-pds/v1-0-0/

This block of code will utilize the AWS bucket and take one day, and one element, and plot it in Python. We have to submit arguments to indicate the day and element we want. **Let's pick a random day of no importance (or pick one yourself)**

In [ ]:
#Insert Arguments Here
year= 2019  
month= 2
day= 3
element= 'prcp' 

This next block will set up some date and plotting info, based on the arguments chosen above.

Feel free to play with the date formatting, color schemes, and bounds I've used. 

<a href='https://colorbrewer2.org/' target='blank'>ColorBrewer</a> is an excellent website to help one figure out color schemes.

In [ ]:
# Get some Info Based on Date
day_counter=int(day)
current_date="%04i%02i%02i" % (int(year),int(month),int(day))
current_month="%04i%02i" % (int(year),int(month))

# Set info based on Element
if element == "tmax" or element == "tmin" or element == "tavg":
    unit='°F'
    if element=='tmax':
        element_name='Maximum Temperature'
    if element=='tmin':
        element_name='Minimum Temperature'    
    if element=='tavg':
        element_name='Average Temperature'

    # Set Up Color Map
    cmap='RdYlBu_r' 
    bounds=np.array([0,10,20,30,40,50,60,70,80,90,100,110])   
        
if element == "prcp":
    unit='inches'  
    element_name='Precipitation'

    # Set Up Color Map
    cmap='YlGnBu' 
    bounds=np.array([0.01,0.10,0.25,0.50,0.75,1.00,2.00,3.00,4.00], dtype='f')

# Set up some other Plotting info
dpi=100

### Ok let's read some data.

nClimGrid Daily is updated each day, so you should be able to get the most recent data. This block of code will attempt to find the necessary data in the AWS bucket. The netCDF files are stored by month. Since this data is publicly available via the <a href="https://www.noaa.gov/information-technology/big-data" target="blank">NOAA Open Data Dissemination (NODD) Project</a>, anonymous data read should be fine (`anon=True`)

Also, since the data is updated frequently, some of the most recent files indicate 'prelim' for preliminary, and finalized versions are noted as 'scaled.' This block of code will read in the data from AWS, knowing this discrepancy.

In [ ]:
#################################################
# READ IN DATA FROM AWS

# Open File System
s3 = s3fs.S3FileSystem(anon=True)

# Get Specific File Based on Inputs
aws_year="%04i" % (int(year))
aws_month="%02i" % (int(month))
aws_version='prelim'
aws_url='s3://noaa-nclimgrid-daily-pds/v1-0-0/grids/'+aws_year+'/ncdd-'+aws_year+aws_month+'-grd-'+aws_version+'.nc'

# Read in Data
print('READING IN nClimGrid DATA FROM AWS: ',aws_url)
try:
    aws_file=s3.open(aws_url)
except:
    aws_version='scaled'
    aws_url='s3://noaa-nclimgrid-daily-pds/v1-0-0/grids/'+aws_year+'/ncdd-'+aws_year+aws_month+'-grd-'+aws_version+'.nc'
    print('FAILED, TRYING THIS URL: ',aws_url)
    aws_file=s3.open(aws_url)

data_monthly = xr.open_dataset(aws_file)
print("\tSUCCESS!\n")

### Examining the data
You may get a warning after reading the data, but if you've made it this far, congrats, you read in data from AWS! 

Well, let's made sure it was read in correctly:

In [ ]:
data_monthly

The netCDF file should have lat (596 points) and lon (1385 points) dimensions, with a time dimension indicating how many days in the month. 

You'll also notice that the lat/lon values are noted in degrees. This will be important later. 

Let's go ahead and get the lat/lon values, as well as the data for a particular day. This data is originally in metric (C and mm) units, so we will convert to imperial units (F and inches) here.

In [ ]:
# Get Lat/Lons
nclimgrid_lats=data_monthly.lat.values
nclimgrid_lons=data_monthly.lon.values

# Get Data, Convert from metric
if element == 'prcp':
    conus_value=(data_monthly[element].values[day_counter-1,:,:] * 0.0393701)  
else:
    conus_value=(data_monthly[element].values[day_counter-1,:,:] * 1.8) + 32.

In [ ]:
conus_value

Gah! Nan's? Well Don't worry, there's data (I promise). To prove it, let's pick one gridpoint

In [ ]:
print(conus_value[300,300])

See? Nothing to Worry About. 

Now for the fun part...

### Plotting the data!

But before we get there, we want YOU to be the creator of the final product (not me!) So let's give credit to yourself:

In [ ]:
author='Not Jared Rennie'

Ok, now let's plot. The following block of code does the following:

- Set up the colorbar
- Create the Figure in the Lambert Conformal Projection
- Adjust the map to fit CONUS
- Add boundaries such as US States
- Plot the data we have aquired
- Add colorbar and other info (including authorship!) 
- Show the results

I would take a good look at this block. This is the result of months of trial and error to make this great! Feel free to tinker (if you're feeling confident)

**PRO TIP**: This took me forever to realize, but since cartopy projections and transformations can be frustrating, I'm going to give you a piece of advice
1. The projection you want the final output to be in is indicated here: 

`ax = fig.add_axes([0, 0, 1, 1], projection=ccrs.LambertConformal())`

2. In order for the data to match to the projection indicated in 1. above, you must indicate what the projection of the data currently is when plotting: 

`ax.pcolormesh(nclimgrid_lons, nclimgrid_lats, conus_value,transform=ccrs.PlateCarree(),cmap=cmap,norm=norm,vmin=vmin,vmax=vmax)`

In other words, we know the data is in a lat/lon projection, so using these two commands indicate that the data is in lat/lon `ccrs.PlateCarree()` and we want to re-project onto the map `ccrs.LanbertConformal()` 

In [ ]:
#################################################
# PLOTTING CONUS
print("PLOTTING CONUS")

# Set Up Colorbar info
vmin=np.min(bounds)
vmax=np.max(bounds)
extend='both'
norm = mcolors.BoundaryNorm(boundaries=bounds, ncolors=256)
cm = plt.cm.ScalarMappable(cmap=cmap)
cm.set_array(np.array(conus_value))
cm.set_clim(vmin, vmax)

# Set CONUS Bounds
minLat = 22    
maxLat = 50   
minLon = -120 
maxLon = -73 

# Set Up Figure
fig= plt.figure(num=1, figsize=(8,5), dpi=dpi, facecolor='w', edgecolor='k')
ax = fig.add_axes([0, 0, 1, 1], projection=ccrs.LambertConformal())
ax.set_extent([minLon, maxLon, minLat, maxLat], crs=ccrs.Geodetic())
ax.set_facecolor('#4a4a4a')

# Add Boundaries
ax.add_feature(cfeature.COASTLINE,linewidth=0.5)
ax.add_feature(cfeature.BORDERS,linewidth=0.5)
ax.add_feature(cfeature.STATES,linewidth=0.5)

# Plot Data
ax.pcolormesh(nclimgrid_lons, nclimgrid_lats, conus_value,transform=ccrs.PlateCarree(),cmap=cmap,norm=norm,vmin=vmin,vmax=vmax)

# Add Colorbar
cax = fig.add_axes([0.1, -0.035, 0.8, 0.03])
cbar=plt.colorbar(cm, cax=cax,boundaries=bounds,orientation='horizontal',extend=extend,spacing='uniform')
cbar.ax.tick_params(labelsize=15)
cbar.set_label(unit,size=15)

# Annotate Info On Graphic
nclimgrid_max="%6.1f" % np.nanmax(conus_value)
nclimgrid_min="%6.1f" % np.nanmin(conus_value)
plt.annotate('Source: nClimGrid Daily\nMade By: '+author,xy=(1.045, -3.51), xycoords='axes fraction', fontsize=7,color='black',horizontalalignment='right', verticalalignment='bottom')
if element == "prcp":
    plt.annotate('MAX: '+str(nclimgrid_max)+'"\nMIN: '+str(nclimgrid_min)+'"',xy=(0.045, -3.51), xycoords='axes fraction', fontsize=7,color='black',horizontalalignment='right', verticalalignment='bottom')
else:
    plt.annotate('MAX: '+str(nclimgrid_max)+'°F\nMIN: '+str(nclimgrid_min)+'°F',xy=(0.045, -3.51), xycoords='axes fraction', fontsize=7,color='black',horizontalalignment='right', verticalalignment='bottom')

# Add Title and Save to File
plt.suptitle('Daily '+element_name+' for '+current_date,size=17,color='black',y=1.05) 
plt.show() 
plt.clf()
plt.close()

Congrats! You made a CONUS plot using NOAA data stored on AWS! Feel free to post on social media for the world to see!

### Challenge: Clipping

Now some of you might be thinking, "Cool Cool, but I live in the Carolinas, and don't want to squint to see the data there. 

Well have no fear, we can do some GIS Style clipping to make it work! All in Python!

First We have to organize the data we read in a little more. We know that the coordinates are in lat/lon coordinate reference system, so we need to indicate that (for the non-GIS folks, it's EPSG 4326). This is where rioxarray comes in.

In [ ]:
# Set Up Spatial Configurations of nClimGrid Data (For Clipping Purposes)
data_monthly=data_monthly.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
data_monthly=data_monthly.rio.write_crs('epsg:4326', inplace=True)

This notebook should have a New England shapefile associated with it. Let's read it in using geopandas

In [ ]:
# Read in Shapefile For Clipping Purposes
shapefile_name='Carolinas'
input_shapefile='input_shapefile/'+str(shapefile_name)+'_Merc.shp'
geo_shapefile = gpd.read_file(input_shapefile)
projection=geo_shapefile.crs

Even though the Shapefile indicates it might be in the Mercator Projection ('Merc'), it doesn't hurt to check

In [ ]:
print(projection)

epsg:3395 is World Mercator. But note the that epsg values are different between the nClimGrid data and the Shapefile (4326 and 3395, respectively). Thankfully because we defined both, this should be taken care of when we clip.

So, let's go ahead and clip and get the data we need.

In [ ]:
# Clip CONUS Data By Shapefile
data_monthly_clip=data_monthly.rio.clip(geo_shapefile.geometry.apply(mapping), geo_shapefile.crs, drop=False)

# Get Data, Convert from metric
if element == 'prcp':
    conus_value=(data_monthly_clip[element].values[day_counter-1,:,:] * 0.0393701)  
else:
    conus_value=(data_monthly_clip[element].values[day_counter-1,:,:] * 1.8) + 32.

Now let's plot again! To prove the clip worked, let's keep the same figure we had (CONUS) and see if New England data is the only part that is plotted.

p.s. for those that are screaming "pUt tHiS iN a fUnCtIoN", I... didn't feel like it. 

In [ ]:
#################################################
# PLOTTING BASED ON SHAPEFILE
print("PLOTTING BASED ON SHAPEFILE")

# Set Up Colorbar info
vmin=np.min(bounds)
vmax=np.max(bounds)
extend='both'
norm = mcolors.BoundaryNorm(boundaries=bounds, ncolors=256)
cm = plt.cm.ScalarMappable(cmap=cmap)
cm.set_array(np.array(conus_value))
cm.set_clim(vmin, vmax)

# Set CONUS Bounds
minLat = 22    
maxLat = 50   
minLon = -120 
maxLon = -73 

# Set Up Figure
fig= plt.figure(num=1, figsize=(8,5), dpi=dpi, facecolor='w', edgecolor='k')
ax = fig.add_axes([0, 0, 1, 1], projection=ccrs.LambertConformal())
ax.set_extent([minLon, maxLon, minLat, maxLat], crs=ccrs.Geodetic())
ax.set_facecolor('#4a4a4a')

# Add Boundaries
ax.add_feature(cfeature.COASTLINE,linewidth=0.5)
ax.add_feature(cfeature.BORDERS,linewidth=0.5)
ax.add_feature(cfeature.STATES,linewidth=0.5)

# Plot Data
ax.pcolormesh(nclimgrid_lons, nclimgrid_lats, conus_value,transform=ccrs.PlateCarree(),cmap=cmap,norm=norm,vmin=vmin,vmax=vmax)

# Add Colorbar
cax = fig.add_axes([0.1, -0.035, 0.8, 0.03])
cbar=plt.colorbar(cm, cax=cax,boundaries=bounds,orientation='horizontal',extend=extend,spacing='uniform')
cbar.ax.tick_params(labelsize=15)
cbar.set_label(unit,size=15)

# Annotate Info On Graphic
nclimgrid_max="%6.1f" % np.nanmax(conus_value)
nclimgrid_min="%6.1f" % np.nanmin(conus_value)
plt.annotate('Source: nClimGrid Daily\nMade By: '+author,xy=(1.045, -3.51), xycoords='axes fraction', fontsize=7,color='black',horizontalalignment='right', verticalalignment='bottom')
if element == "prcp":
    plt.annotate('MAX: '+str(nclimgrid_max)+'"\nMIN: '+str(nclimgrid_min)+'"',xy=(0.045, -3.51), xycoords='axes fraction', fontsize=7,color='black',horizontalalignment='right', verticalalignment='bottom')
else:
    plt.annotate('MAX: '+str(nclimgrid_max)+'°F\nMIN: '+str(nclimgrid_min)+'°F',xy=(0.045, -3.51), xycoords='axes fraction', fontsize=7,color='black',horizontalalignment='right', verticalalignment='bottom')

# Add Title and Save to File
plt.suptitle('Daily '+element_name+' for '+current_date,size=17,color='black',y=1.05) 
plt.show() 
plt.clf()
plt.close()

Double check this plot vs the previous, but they should be the same (well, at least for New England)

Also note that because we clipped the data, the Max/Min values on the bottom left are different.

Now let's plot one more time, but change the map projection to Mercator, and zoom in on New England!

In [ ]:
# Set Shapefile Bounds
minLat = 40    
maxLat = 48   
minLon = -74 
maxLon = -66 

# Set Up Figure
fig= plt.figure(num=1, figsize=(8,8), dpi=dpi, facecolor='w', edgecolor='k')
ax = fig.add_axes([0, 0, 1, 1], projection=ccrs.Mercator())
ax.set_extent([minLon, maxLon, minLat, maxLat], crs=ccrs.Geodetic())
ax.set_facecolor('#4a4a4a')

# Add Boundaries
ax.add_feature(cfeature.COASTLINE,linewidth=0.5)
ax.add_feature(cfeature.BORDERS,linewidth=0.5)
ax.add_feature(cfeature.STATES,linewidth=0.5)

# Plot Data
ax.pcolormesh(nclimgrid_lons, nclimgrid_lats, conus_value,transform=ccrs.PlateCarree(),cmap=cmap,norm=norm,vmin=vmin,vmax=vmax)

# Add Colorbar
cax = fig.add_axes([0.1, -0.035, 0.8, 0.03])
cbar=plt.colorbar(cm, cax=cax,boundaries=bounds,orientation='horizontal',extend=extend,spacing='uniform')
cbar.ax.tick_params(labelsize=15)
cbar.set_label(unit,size=15)

# Annotate Info On Graphic
nclimgrid_max="%6.1f" % np.nanmax(conus_value)
nclimgrid_min="%6.1f" % np.nanmin(conus_value)
plt.annotate('Source: nClimGrid Daily\nMade By: '+author,xy=(1.045, -3.51), xycoords='axes fraction', fontsize=7,color='black',horizontalalignment='right', verticalalignment='bottom')
if element == "prcp":
    plt.annotate('MAX: '+str(nclimgrid_max)+'"\nMIN: '+str(nclimgrid_min)+'"',xy=(0.045, -3.51), xycoords='axes fraction', fontsize=7,color='black',horizontalalignment='right', verticalalignment='bottom')
else:
    plt.annotate('MAX: '+str(nclimgrid_max)+'°F\nMIN: '+str(nclimgrid_min)+'°F',xy=(0.045, -3.51), xycoords='axes fraction', fontsize=7,color='black',horizontalalignment='right', verticalalignment='bottom')

# Add Title and Save to File
plt.suptitle('Daily '+element_name+' for '+current_date,size=17,color='black',y=1.05) 
plt.show() 
plt.clf()
plt.close()

Isn't it beautiful?

**Congrats on completing this notebook! Now go forth and make better plots!**

### Some other challenges you could try
- One in theory could calculate diurnal temperature range, which is max minus min
- How could one add county lines, or point data indicating cities. Could other shapefiles be added as layers?
- There's other gridded data publicly available from the NOAA Big Data Project. Could one plot that as well?